In [137]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import xlsxwriter
import datetime


In [138]:
# ususaarios de base de datos local

usuario = "care_consulta"
contrasena = "CareAssistance!10"
url_servidor = "10.0.10.22"
# puerto = '3306'
# esquema = 'EspoCRM2'
# plugin_autenticacion = 'mysql_native_password'

# usuarios de base de datos Simtastic ##########################################

# usuario = 'siempreread'
##contrasena = 'S13mpr3_Ch1l3'
# url_servidor = '209.151.148.250'
puerto = "3306"
esquema = "EspoCRM2"
plugin_autenticacion = "mysql_native_password"

# Conexión a la base de datos

c_conexion = "mysql+mysqlconnector://{0}:{1}@{2}:{3}/{4}?auth_plugin={5}"
c_conexion = c_conexion.format(
    usuario, contrasena, url_servidor, puerto, esquema, plugin_autenticacion
)
motor_mysql_mariadb = create_engine(c_conexion)

In [139]:
sql = "select cas.number as 'Id',trim(con.rut) as 'Rut Paciente',trim(con.rut_asegurado) as 'Rut Titular',con.relacion_con_asegurado as 'Relación con asegurado', cas.area as 'Prestacion',"
sql2 = " cas.created_at as 'Fecha Gestión', con.empleador AS 'Empleador', cas.channel as 'Canal', trim(con.campana_paciente) as 'Campaña'"
sql3 = "from EspoCRM2.case cas left join EspoCRM2.contact con on cas.contact_id = con.id where con.campana_paciente like '%metlife%' AND cas.deleted = 0 and cas.status <> 'Cancelled' and con.empleador like '%concha%' and cas.created_at > '2022-11-01' order by cas.number desc;"

fin_sql = sql + sql2 + sql3


In [140]:
conchaytoro = pd.read_sql_query(fin_sql, motor_mysql_mariadb)


# archivo de salida modificar segun archivo
archivo = "C:/Users/jmartinez/Desktop/Informes/CYT/Concha y Toro.xlsx"
conchaytoro.head(10)

,Id,Rut Paciente,Rut Titular,Relación con asegurado,Prestacion,Fecha Gestión,Empleador,Canal,Campaña
0,270170,26745138-9,26745093-5,HI,Siempre - Medicina,2022-11-30 10:39:32,conchaytoro,Llamada telefónica,Metlife
1,269982,12460285-8,12460285-8,AS,Siempre - Medicina,2022-11-24 11:51:10,conchaytoro,Video Call,Metlife
2,269930,27076917-9,27076917-9,AS,Siempre - Front,2022-11-23 18:56:12,conchaytoro,WhatsApp,Metlife
3,269656,12460285-8,12460285-8,AS,Siempre - Medicina,2022-11-22 18:10:22,conchaytoro,Video Call,Metlife
4,269577,26639577-9,26862659-k,HI,Siempre - Nutrición,2022-11-22 15:00:48,conchaytoro,Video Call,METLIFE
5,269576,26639577-9,26862659-k,HI,Siempre - Nutrición,2022-11-22 15:00:20,conchaytoro,Video Call,METLIFE
6,269510,18469843-9,18469843-9,AS,Siempre - Front,2022-11-22 13:05:20,conchaytoro,Chat Online,Metlife
7,269465,26763022-4,16089751-1,HI,Siempre - Medicina,2022-11-22 02:40:02,conchaytoro,WhatsApp,Metlife
8,269086,26639577-9,26862659-k,HI,Siempre - Agenda,2022-11-21 13:51:49,conchaytoro,Llamada telefónica,METLIFE
9,268698,18539658-4,18539658-4,AS,Siempre - Nutrición,2022-11-18 19:09:12,conchaytoro,Llamada telefónica,Metlife


In [141]:
conchaytoro.shape


(89, 9)

In [142]:
relacion = {"HI": "Hijo/a", "AS": "Titular", "CO": "Cónyuge"}
conchaytoro.replace({"Relación con asegurado": relacion}, inplace=True)

prestaciones = {
    "Deporte": "Programa Deportivo",
    "Nutrición": "Programa Nutricional",
    "Pcte. Crónico": "Programa Enfermería",
    "Orientación": "Orientación Hospitalaria",
    "Psicología": "Programa de Psicología",
    "Psicología": "Programa de Psicología",
    "Medicina": "Doctor Online",
    "Deporte": "Programa Deportivo",
    "Pcte. Crónico": "Programa de Hospitalización Domiciliaria",
    "Pcte. Crónico": "Programa de Hospitalización Domiciliaria",
    "Medicina": "Doctor Online",
    "Psicología": "Programa de Psicología",
    "Nutrición": "Programa Nutricional",
    "Deporte": "Programa Deportivo",
    "Fidelización": "Fidelización",
    "Orientación": "Orientación Hospitalaria",
    "Pcte. Crónico": "Programa Enfermería",
    "Medicina": "Doctor Online",
    "Agenda": "Orientación Hospitalaria",
    "Fidelización y SAC": "Orientación Hospitalaria",
    "Ingreso dep.  seguro": "Programa Deportivo",
    "Clínica del Sueño": "Programa Enfermería",
    "Ingreso dep. seguro": "Programa Deportivo",
    "APSSex": "Programa de Psicología",
    "Kinesiología HC": "Programa Deportivo",
    "Siempre - Medicina": "Doctor Online",
    "Siempre - Orientación": "Orientación Hospitalaria",
    "Siempre - Agenda": "Orientación Hospitalaria",
    "Siempre - Deportiva": "Programa Deportivo",
    "Siempre - Psicología": "Programa de Psicología",
    "Siempre - Nutrición": "Programa Nutricional",
    "Siempre  - Front": "Orientación Hospitalaria",
    "Siempre - Clínica del Sueño": "Programa Enfermería",
    "Siempre - Enfermería": "Programa Enfermería",
    "Telemedicina": "Doctor Online",
    "Nutricion": "Programa Nutricional",
    "Front": "Orientación Hospitalaria",
    "Deportivo": "Programa Deportivo"
}
conchaytoro.replace({"Prestacion": prestaciones}, inplace=True)



conchaytoro['derty'] = pd.to_datetime(conchaytoro['Fecha Gestión'], format="%d-%m-%Y %H:%M")
#conchaytoro.derty = conchaytoro.derty.dt.strftime("%d-%m-%Y %H:%M")
#conchaytoro['Fecha Gestión'] = conchaytoro['Fecha Gestión'].apply(lambda _: datetime.strptime(_,"%d-%m-%Y, %H:%M"))

In [143]:
conchaytoro.head(50)


,Id,Rut Paciente,Rut Titular,Relación con asegurado,Prestacion,Fecha Gestión,Empleador,Canal,Campaña,derty
0,270170,26745138-9,26745093-5,Hijo/a,Doctor Online,2022-11-30 10:39:32,conchaytoro,Llamada telefónica,Metlife,2022-11-30 10:39:32
1,269982,12460285-8,12460285-8,Titular,Doctor Online,2022-11-24 11:51:10,conchaytoro,Video Call,Metlife,2022-11-24 11:51:10
2,269930,27076917-9,27076917-9,Titular,Orientación Hospitalaria,2022-11-23 18:56:12,conchaytoro,WhatsApp,Metlife,2022-11-23 18:56:12
3,269656,12460285-8,12460285-8,Titular,Doctor Online,2022-11-22 18:10:22,conchaytoro,Video Call,Metlife,2022-11-22 18:10:22
4,269577,26639577-9,26862659-k,Hijo/a,Programa Nutricional,2022-11-22 15:00:48,conchaytoro,Video Call,METLIFE,2022-11-22 15:00:48
...,...,...,...,...,...,...,...,...,...,...
45,266306,15654189-3,15654189-3,Titular,Doctor Online,2022-11-12 18:58:49,conchaytoro,Llamada telefónica,Metlife,2022-11-12 18:58:49
46,265878,26639577-9,26862659-k,Hijo/a,Orientación Hospitalaria,2022-11-11 16:54:24,conchaytoro,Llamada telefónica,METLIFE,2022-11-11 16:54:24
47,265831,14159469-9,15898174-2,Cónyuge,Programa Nutricional,2022-11-11 16:05:07,conchaytoro,Llamada telefónica,Metlife,2022-11-11 16:05:07
48,265666,18119050-7,18119050-7,Titular,Programa Nutricional,2022-11-11 12:52:33,conchaytoro,Correo electrónico,Metlife,2022-11-11 12:52:33


In [144]:
conchaytoro.to_excel(archivo,index=False)